# Исследование надежности заемщиков

# Описание проекта

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

# Описание данных
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита


## Шаг 1. Обзор данных

In [1]:
import pandas as pd


In [2]:
data = pd.read_csv('./data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


## Шаг 2. Предобработка данных

### Шаг 2.1 Заполнение пропусков

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Обнаружила, что пропущенные данные имеются в days_employed и total_income. Их количество составляет 2174.

In [5]:
(data.isna().sum() / data.isna().count()) * 100

children             0.000000
days_employed       10.099884
dob_years            0.000000
education            0.000000
education_id         0.000000
family_status        0.000000
family_status_id     0.000000
gender               0.000000
income_type          0.000000
debt                 0.000000
total_income        10.099884
purpose              0.000000
dtype: float64

Пропущенные значения составляют 10.1% от общего количества.

In [6]:
data[data['total_income'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


In [7]:
data[(data['total_income'].isna()) & (data['days_employed'].isna() == False)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Т.к. пустые значения обоих столбцов совпадают, то предполагаю, что данные отсутствуют, т.к. не было данных о трудовом стаже этого человека, соответственно и его зарплаты. Возможно данные люди оформляли банковские услуги без подтверждения дохода и общего стажа.

In [8]:
median_ti = data['total_income'].median()
data['total_income'] = data['total_income'].fillna(median_ti)

При вычислении средней зарплаты лучше применять метод median(), т.к. среднее значение может сильно зависеть от очень больших или малых значений столбца. Медиана в этом случае показывает более "реальный" средний результат.

### Шаг 2.2 Проверка данных на аномалии и исправления

In [9]:
def is_nan(value):
    return value != value

def get_anomaly(df):
    if is_nan(df['days_employed']):
        return "Пустое значение"
    years_employed = df['days_employed'] / 365
    if years_employed > df['dob_years']:
        return "Количество отработанных лет больше, чем возраст"
    if years_employed < 0:
        if abs(years_employed) < df['dob_years']:
            return "Количество отработанных лет отрицательно, но в абсолюте не завышено"
        else:
            return "Количество отработанных лет отрицательно, в абсолюте завышено"
    else:
        return "Данные корректны"

In [10]:
data['anomaly'] = data.apply(get_anomaly, axis = 1)
data.groupby('anomaly')['gender'].count()

anomaly
Количество отработанных лет больше, чем возраст                         3445
Количество отработанных лет отрицательно, в абсолюте завышено             74
Количество отработанных лет отрицательно, но в абсолюте не завышено    15832
Пустое значение                                                         2174
Name: gender, dtype: int64

Обнаружила, что все данные ошибочны. Присутствуют такие ошибки, как пустые значения, отрицательные значения, отрицательные значения, которые превышают возвраст человека и также значения, снова превышающие возраст человека.
Такие ошибки могут возникнуть, потому что:
1. Отрицательные значения могут являться положительными значениями, но введенными с ошибкой. В этому случае исправить ошибку можно изменив знак по модулю.
2. Завышенные значения могут быть ошибкой ввода. Исправить их возможным не представляется, т.к. неизвестно, какие данные предполагались при вводе, можно заменить их на медиану.
3. Пустые значения могли возникнуть по разным причинам. Можно заменить их на медианное значение, как уже было сделано ранее.

In [11]:
data.loc[data['anomaly'] == 'Количество отработанных лет отрицательно, но в абсолюте не завышено', 'days_employed'] = data.loc[data['anomaly'] == 'Количество отработанных лет отрицательно, но в абсолюте не завышено', 'days_employed'] * -1
data['anomaly'] = data.apply(get_anomaly, axis = 1)

Исправила ошибку "Количество отработанных лет отрицательно, но в абсолюте не завышено", чтобы иметь предположительно корректные данные.

In [12]:
median_de = data.loc[data['anomaly'] == 'Данные корректны', 'days_employed'].median()
data.loc[data['anomaly'] == 'Количество отработанных лет больше, чем возраст', 'days_employed'] = median_de
data.loc[data['anomaly'] == 'Количество отработанных лет отрицательно, в абсолюте завышено', 'days_employed'] = median_de
data['days_employed'] = data['days_employed'].fillna(median_de)
data['anomaly'] = data.apply(get_anomaly, axis = 1)
data.groupby('anomaly')['gender'].count()

anomaly
Данные корректны                                   21424
Количество отработанных лет больше, чем возраст      101
Name: gender, dtype: int64

Исправила такие ошибки, как 'Количество отработанных лет больше, чем возраст', 'Количество отработанных лет отрицательно, в абсолюте завышено' и 'Пустое значение', заменила на медианное значение. 

In [13]:
data['dob_years'].sort_values().unique()

array([ 0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75], dtype=int64)

При исправлении обнаружила, что некоторые значения столбца dob_years равны нулю, что тоже является аномалией.

In [14]:
income_types = data['income_type'].unique()
for it in income_types:
    median = data[data['income_type'] == it]['dob_years'].median()
    data.loc[(data['income_type'] == it) & (data['dob_years'] == 0), 'dob_years'] = median
data['dob_years'] = data['dob_years'].astype('int')

In [15]:
data['dob_years'].sort_values().unique()

array([19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52,
       53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
       70, 71, 72, 73, 74, 75])

Заменила нулевые значения на средние по типу занятости, т.к. тип занятости лучше всего отражает возрастную категорию.

In [16]:
data['children'].sort_values().unique()

array([-1,  0,  1,  2,  3,  4,  5, 20], dtype=int64)

Также обнаружила аномалию в столбце 'children'. Очевидно, аномальными являются значения -1 и 20.

In [17]:
data.loc[data['children'] == 20, 'children'] = 2
data.loc[data['children'] == -1, 'children'] = 1

In [18]:
data['children'].sort_values().unique()

array([0, 1, 2, 3, 4, 5], dtype=int64)

Это все аномалии, которые я обнаружила.

### Шаг 2.3 Изменение типов данных

In [19]:
data['total_income'] = data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int32  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int32  
 11  purpose           21525 non-null  object 
 12  anomaly           21525 non-null  object 
dtypes: float64(1), int32(2), int64(4), object(6)
memory usage: 2.0+ MB


### Шаг 2.4 Удаление дубликатов

In [20]:
data = data.drop_duplicates().reset_index(drop=True)

In [21]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [22]:
data['education'] = data['education'].str.lower()

In [23]:
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [24]:
data['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

In [25]:
data = data[data['gender'] != 'XNA']

In [26]:
data['gender'].unique()

array(['F', 'M'], dtype=object)

Явные дубликаты я удалила через drop_duplicates, а также обновила индексацию с помощью метода reset_index(). Для поиска неявных дубликатов вывела уникальные значения столбцов, в которых могли повторяться значения, с помощью метода unique(), а после этого данные либо удалила, если они не имели критического значения, либо привела к единому регистру с помощью метода str.lower(), т.к. в столбце "education" значения были указаны правильно, только имели прописные буквы.

Возможные причины появления дубликатов:
1. Человеческий фактор. В случае с образованием разные люди вводили данные в разном стиле.
2. Причины полных дубликатов строк может быть связана с ошибкой вводы или же с повторными заявками.
3. С полом человека можно предположить, что запись была введена ошибочно.

### Шаг 2.5 Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма

In [27]:
data_edu = data[['education','education_id']].drop_duplicates()
data_edu

,education,education_id
0,высшее,0
1,среднее,1
13,неоконченное высшее,2
31,начальное,3
2962,ученая степень,4


In [28]:
data_fs = data[['family_status','family_status_id']].drop_duplicates()
data_fs

,family_status,family_status_id
0,женат / замужем,0
4,гражданский брак,1
18,вдовец / вдова,2
19,в разводе,3
24,Не женат / не замужем,4


In [29]:
data = data.drop(columns=['education','family_status'], axis=1)
data

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,anomaly
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,Данные корректны
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,Данные корректны
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,Данные корректны
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,Данные корректны
4,0,1630.691372,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,Данные корректны
...,...,...,...,...,...,...,...,...,...,...,...
21466,1,4529.316663,43,1,1,F,компаньон,0,224791,операции с жильем,Данные корректны
21467,0,1630.691372,67,1,0,F,пенсионер,0,155999,сделка с автомобилем,Данные корректны
21468,1,2113.346888,38,1,1,M,сотрудник,1,89672,недвижимость,Данные корректны
21469,3,3112.481705,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля,Данные корректны


### Шаг 2.6 Категоризация дохода

In [30]:
def total_income_category(total_income):
    if 0 < total_income <= 30000:
        return 'E'
    if 30001 < total_income <= 50000:
        return 'D'
    if 50001 < total_income <= 200000:
        return 'C'
    if 200001 < total_income <= 1000000:
        return 'B'
    else:
        return 'A'

data['total_income_category'] = data['total_income'].apply(total_income_category)

### Шаг 2.7 Категоризация целей кредита

In [31]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [32]:
def purpose_category(purpose):
    if 'автомобил' in purpose:
        return 'операции с автомобилем'
    if 'жиль' in purpose or 'недвижимост' in purpose:
        return 'операции с недвижимостью'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    return 'получение образования'
    
    
data['purpose_category'] = data['purpose'].apply(purpose_category)

### Ответы на вопросы:

##### Вопрос 1:  Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [33]:
data_pivot = data.pivot_table(index=['children'], columns='debt', values='purpose', aggfunc='count')
data_pivot

debt,0,1
children,,
0,13043.0,1063.0
1,4411.0,445.0
2,1926.0,202.0
3,303.0,27.0
4,37.0,4.0
5,9.0,NaN


In [34]:
(data_pivot[1].fillna(0) / (data_pivot[0] + data_pivot[1].fillna(0))) * 100

children
0    7.535800
1    9.163921
2    9.492481
3    8.181818
4    9.756098
5    0.000000
dtype: float64

##### Вывод 1:

*Зависимость от количества детей есть, но неявная. Можно заметить разницу в долях между теми, у кого дети есть и у кого их вовсе нет, и также можно заметить небольшой прирост в доле должников по мере роста количества детей. С другой стороны, этот небольшой прирост слишком мал, чтобы считать его значимым.*

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [35]:
data_pivot1 = data.merge(data_fs, on='family_status_id', how='left').pivot_table(index=['family_status'], columns='debt', values='purpose', aggfunc='count')
data_pivot1

debt,0,1
family_status,,
Не женат / не замужем,2536,274
в разводе,1110,85
вдовец / вдова,896,63
гражданский брак,3774,388
женат / замужем,11413,931


In [36]:
(data_pivot1[1] / (data_pivot1[0] + data_pivot1[1])) * 100

family_status
Не женат / не замужем    9.750890
в разводе                7.112971
вдовец / вдова           6.569343
гражданский брак         9.322441
женат / замужем          7.542126
dtype: float64

##### Вывод 2:

*На основании полученных данных явная зависимость не прослеживается. Вижу, что доля тех, кто женат / замужем или в гражданском браке примерно совпадает с теми, кто не женат / не замужем или в разводе.*

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [37]:
data_pivot2 = data.pivot_table(index=['total_income_category'], columns='debt', values='purpose', aggfunc='count')
data_pivot2

debt,0,1
total_income_category,,
A,24,2
B,4683,356
C,14673,1360
D,329,21
E,20,2


In [38]:
(data_pivot2[1] / (data_pivot2[0] + data_pivot2[1])) * 100

total_income_category
A    7.692308
B    7.064894
C    8.482505
D    6.000000
E    9.090909
dtype: float64

##### Вывод 3:

*На первый взгляд можно заметить зависимость, но с другой стороны невозможно утверждать о ее наличии, т.к. во всех категориях сильно отличается объем данных, и замеченная зависимость может оказаться ложной при дальнейшем анализе этих данных.*

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

In [39]:
data_pivot3 = data.pivot_table(index=['purpose_category'], columns='debt', values='purpose', aggfunc='count')
data_pivot3

debt,0,1
purpose_category,,
операции с автомобилем,3905,403
операции с недвижимостью,10031,782
получение образования,3644,370
проведение свадьбы,2149,186


In [40]:
(data_pivot3[1] / (data_pivot3[0] + data_pivot3[1])) * 100

purpose_category
операции с автомобилем      9.354689
операции с недвижимостью    7.232036
получение образования       9.217738
проведение свадьбы          7.965739
dtype: float64

##### Вывод 4:

*Операции с автомобилем и получение образование менее возвращаемы, чем операции с недвижимостью и проведение свадьбы.*

## Общий вывод:

**Из проведенного выше анализа можно сделать следующие выводы:**
+ Зависимость от наличия детей есть, при этом зависимость от количества детей неявная.
+ Зависимость от семейного положения не прослеживается.
+ При вычислении зависимости от уровня дохода сильное различие в объеме данных не позволяет дать корректную оценку.
+ Операции с автомобилем и получение образование менее возвращаемы, чем операции с недвижимостью и проведение свадьбы.

**Общий вывод: при выдаче кредита прдпочтения можно отдавать людям без детей на покупку недвижимости и проведение свадьбы.**